In [1]:
"""
Created on Wed Jul 12 10:27:50 2017

@author: yunyuan
# add ploting signal delay 2017-07-11
# add curve fitting 2017-07-20
# merge wang 2017-07-23
"""
from PyStartClient import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
import seaborn as sns
import numpy as np
import math
import scipy as sp
from datetime import timedelta


client = PyStartClient("http://211.87.5.13/suzhouservice")
startTime = datetime.datetime(2017, 4, 11, 8, 00, 0)
endTime = datetime.datetime(2017, 4, 11, 8, 30, 0)
linkID="62-3"
cyleInfoList = client.getLinkCycleDetailsInfo(linkID, startTime, endTime)

In [2]:
index = -1
stageIndex=2
m=3800
freett=37

lgId=linkID+'-137-5'
#lgId=linkID+'-63-1'
#lgId=linkID+'-66-1'
#lgId=linkID+'-64-5'
for i, lgInfo in enumerate(cyleInfoList):
    #print(lgInfo.LgId) #whos LgId
    if lgId == lgInfo.LgId:
        index = i
        break


#    if index == -1:
#        print('lgId not found')
#        return

#---plot cycle figure from here
stime=[x.StartTime for x in cyleInfoList[index].CycleInfoList]
count=[x.Count for x in cyleInfoList[index].CycleInfoList]
perCount=[x.Count/x.Period for x in cyleInfoList[index].CycleInfoList]
cycle=[x.Period for x in cyleInfoList[index].CycleInfoList]
lamda=[x.CycleStageList[1].StageSeconds/x.Period for x in cyleInfoList[index].CycleInfoList]
stage=[x.CycleStageList[stageIndex].StageSeconds for x in cyleInfoList[index].CycleInfoList]
stagestart=[x.CycleStageList[stageIndex].StartTime for x in cyleInfoList[index].CycleInfoList]
stageend=[x.CycleStageList[stageIndex].EndTime for x in cyleInfoList[index].CycleInfoList]
#stage=[x.CycleStageList[1].StageSeconds for x in cyleInfoList[index].CycleInfoList]
#stage=stage/cycle
flow=[x.Count*3600/x.Period for x in cyleInfoList[index].CycleInfoList]
validcount=[x.ValidCount for x in cyleInfoList[index].CycleInfoList]
traveltime=[x.AverageTravelTime for x in cyleInfoList[index].CycleInfoList]
signaldelay=[x.AverageSignalDelay for x in cyleInfoList[index].CycleInfoList]
saturation=[x.Count*3600/m/x.CycleStageList[1].StageSeconds for x in cyleInfoList[index].CycleInfoList]
#percount=[x.Count/x.Period*100 for x in cyleInfoList[index].CycleInfoList]
#print(np.var([x/y for x,y in zip(validcount,count)]))

In [3]:
plt.figure(1,figsize=(8,4))
plt.plot(stime,cycle,'bo')
plt.xlabel('Cycle Timestamp')
plt.ylabel('Cycle Length (s)')
plt.savefig('Timestamp-Cycle length.png',dpi=600,bbox_inches='tight')
plt.show()

In [4]:
plt.figure(2,figsize=(8,4))
plt.plot(stime,stage,'b')
plt.xlabel('Cycle Timestamp')
plt.ylabel('Stage Length(s)')
plt.savefig('Timestamp-Stagelength.png',dpi=600,bbox_inches='tight')
plt.show()

In [5]:
plt.figure(3,figsize=(8,4))
plt.plot(stime,count,'b',label='Cycle Count')
plt.plot(stime,validcount,'r',label='Cycle Valid Count')
plt.xlabel('Cycle Timestamp')
plt.ylabel('Vechicle Count(s)')
plt.title('Cycle count vs Cycle valid count')
plt.legend()
plt.savefig('Cycle count vs Cycle valid count.png',dpi=600,bbox_inches='tight')
plt.show()

In [6]:
plt.figure(4,figsize=(8,4))
plt.plot(stime,perCount,'b')
plt.xlabel('Cycle Timestamp')
plt.ylabel('Traffic flow(veh/s)')
plt.savefig('Timestamp-Flowpersecond.png',dpi=600,bbox_inches='tight')
plt.show()

In [7]:
plt.figure(5,figsize=(8,4))
plt.plot(stime,flow,'b')
plt.xlabel('Cycle starting timestamp')
plt.ylabel('Departure flow (veh/h)')
plt.savefig('Cycle departure flow.png',dpi=600,bbox_inches='tight')
print(np.std(flow)/np.mean(flow))
plt.show()

In [8]:
plt.figure(6,figsize=(8,4))
plt.plot(stime,saturation,'b')
plt.xlabel('Cycle timestamp')
plt.ylabel('Saturation rate')
plt.savefig('Timestamp-Cycle saturation.png',dpi=600,bbox_inches='tight')
plt.show()

def getcc(ts,cc,ct):
    i=0
    j=0
    delta=2
    count=1
    sum=0
    
    while(i<len(ts)):
        current=ts[i]
        j=i+1
        
        while(j<len(ts)):
            if (ts[j]-current).seconds<delta:
                count+=1
                j+=1
            else:
                sum+=count;
                #print(str(sum)+" "+current.strftime('%Y-%m-%dT%H:%M:%S'))
                cc.append(sum)
                ct.append(current)
                count=1
                i=j-1
                break
        i+=1

In [9]:
plt.figure(7,figsize=(8,4))

ts=[] 
for y in cyleInfoList[index].CycleInfoList:
    if y.VehTravelTimeList is not None:
        if ts==[]:                            
            ts=[x.TimeStamp for x in y.VehTravelTimeList]
        else:           
            ts.extend([x.TimeStamp for x in y.VehTravelTimeList])             

ct=[]
cc=[]
getcc(ts,cc,ct)

pts=[]
for y in cyleInfoList[index].CycleInfoList:
    if y.VehTravelTimeList is not None:
        if pts==[]:                            
            pts=[x.PreTimeStamp+timedelta(seconds=freett)  for x in y.VehTravelTimeList]
        else:           
            pts.extend([x.PreTimeStamp+timedelta(seconds=freett) for x in y.VehTravelTimeList])


#pts.sort();
pcc=[]
pct=[]   
getcc(pts,pcc,pct)


plt.plot(ct,cc,'r',label='Departure')
plt.plot(pct,pcc,'b',label='Arrival')
#arriving flow

plt.xlabel('Timestamp')
plt.ylabel('Cumulative departure count (veh)')
plt.legend()
plt.savefig('CumulativeArrivalDeparture.png',dpi=600,bbox_inches='tight')
plt.show()

In [10]:
plt.figure(8,figsize=(8,4))
plt.plot(stime,traveltime,'bo')
plt.xlabel('Cycle Timestamp')
plt.ylabel('Travel Time')
plt.title('Average Travel Time '+lgId)
plt.savefig('AverageTravelTime '+lgId+'.png',dpi=600,bbox_inches='tight')
plt.show()

In [11]:
plt.figure(9,figsize=(8,4))
plt.plot(stime,signaldelay,'bo')
plt.xlabel('Cycle Timestamp')
plt.ylabel('Signal Delay')
plt.title('Average Signal Delay '+lgId)
plt.savefig('AverageSignalDelay '+lgId+'.png',dpi=600,bbox_inches='tight')
plt.show()
print(np.mean(signaldelay))

In [12]:
plt.figure(10,figsize=(8,4))
plt.plot(count,traveltime,'bo')
plt.xlabel('Count')
plt.ylabel('Signal Delay')
plt.title('Count vs Delay '+lgId)
plt.savefig('Count vs Delay '+lgId+'.png',dpi=600,bbox_inches='tight')
plt.show()

In [13]:
plt.figure(11,figsize=(8,4))
cmap = sns.blend_palette(["steelblue","yellow","red"], len(cyleInfoList[index].CycleInfoList)) 
sns.set_palette(cmap)
with sns.color_palette(n_colors=len(cyleInfoList[index].CycleInfoList)): 
    for y in cyleInfoList[index].CycleInfoList:
        if y.VehTravelTimeList:
            ttincycle=[x.TravelTime for x in y.VehTravelTimeList]
            #print(ttincycle)
            travelTimeCount = len(ttincycle)
            if travelTimeCount<2 or (travelTimeCount==2 and ttincycle[0] == ttincycle[1]):
                pass    
            else:
                sns.kdeplot(np.array(ttincycle),label=y.StartTime.time())
                #sns.kdeplot(np.array(ttincycle))

plt.xlabel('Travel Time')
plt.ylabel('Frequency')
plt.title('Travel Time Distribution')
plt.savefig('TravelTimeDistribution.png',dpi=600,bbox_inches='tight')
plt.show()

In [14]:
plt.figure(12,figsize=(16,8))
tt=[]
ts=[]
for y in cyleInfoList[index].CycleInfoList:
    if y.VehTravelTimeList is not None:
        if tt==[]:
            tt=[x.TravelTime for x in y.VehTravelTimeList]
            ts=[x.TimeStamp for x in y.VehTravelTimeList]
        else:
            tt.extend([x.TravelTime for x in y.VehTravelTimeList])
            ts.extend([x.TimeStamp for x in y.VehTravelTimeList])


plt.plot(ts,tt,'b.',label='departure time')
#plt.plot(stime,[c+freett for c in cycle],'r')

#arriving flow
freett=48
#自由流行程时间48s
ttt=[]
tts=[]
for y in cyleInfoList[index].CycleInfoList:
    if y.VehTravelTimeList is not None:
        if ttt==[]:
            ttt=[x.TravelTime for x in y.VehTravelTimeList]
            tts=[x.PreTimeStamp+timedelta(seconds=freett) for x in y.VehTravelTimeList]
        else:
            ttt.extend([x.TravelTime for x in y.VehTravelTimeList])
            tts.extend([x.PreTimeStamp+timedelta(seconds=freett) for x in y.VehTravelTimeList])        

plt.plot(tts,ttt,'r.',label='arriving time')
plt.xlabel('Timestamp')
plt.ylabel('Travel Time')
plt.title('Travel Time Each Veh '+lgId)
plt.legend()
plt.savefig('TravelTimeEachVeh.png',dpi=600,bbox_inches='tight')
plt.show()

In [15]:
plt.figure(13,figsize=(16,8))
tt=[]
ts=[]
for y in cyleInfoList[index].CycleInfoList:
    if y.VehTravelTimeList is not None:
        if tt==[]:
            tt=[x.TravelTime for x in y.VehTravelTimeList]
            ts=[x.TimeStamp for x in y.VehTravelTimeList]
        else:
            tt.extend([x.TravelTime for x in y.VehTravelTimeList])
            ts.extend([x.TimeStamp for x in y.VehTravelTimeList])


plt.plot(ts,tt,'d-',label='departure time')
#stage
sig_y=min(tt)-20 
for x,y in zip(stagestart,stageend):
    plt.plot([x,y], [sig_y, sig_y], linewidth=5, color='g')


freett=48
ttt=[]
tts=[]
for y in cyleInfoList[index].CycleInfoList:
    if y.VehTravelTimeList is not None:
        if ttt==[]:
            ttt=[x.TravelTime for x in y.VehTravelTimeList]
            tts=[x.PreTimeStamp+timedelta(seconds=freett) for x in y.VehTravelTimeList]
        else:
            ttt.extend([x.TravelTime for x in y.VehTravelTimeList])
            tts.extend([x.PreTimeStamp+timedelta(seconds=freett) for x in y.VehTravelTimeList])        

plt.plot(tts,ttt,'ms-',label='arriving time')
plt.xlabel('Time')
plt.ylabel('Travel Time (s)')
plt.title('Travel Time & Stage '+lgId)
plt.legend()
plt.savefig('TravelTimeEachVehvsStage.png',dpi=600,bbox_inches='tight')
plt.show()

In [16]:
plt.figure(14,figsize=(16,8))
tt=[]
ts=[]
for y in cyleInfoList[index].CycleInfoList:
    if y.VehTravelTimeList is not None:
        if tt==[]:
            tt=[x.TravelTime for x in y.VehTravelTimeList]
            ts=[x.TimeStamp for x in y.VehTravelTimeList]
        else:
            tt.extend([x.TravelTime for x in y.VehTravelTimeList])
            ts.extend([x.TimeStamp for x in y.VehTravelTimeList])


tt=[t-freett for t in tt]            
plt.plot(ts,tt,'b.',label='departure time')

plt.plot(stime,[c-s for c,s in zip(cycle,stage)],'r',label='cycle-green')
plt.plot(stime,[c for c,s in zip(cycle,stage)],'r--',label='cycle')
plt.xlabel('Timestamp')
plt.ylabel('Delay')
plt.title('Delay Each Veh '+lgId)
plt.legend()
plt.savefig('SecondStop.png',dpi=600,bbox_inches='tight')
plt.show()

In [17]:
plt.figure(15,figsize=(16,8))
tt=[]
ts=[]
for y in cyleInfoList[index].CycleInfoList:
    if y.VehTravelTimeList is not None:
        if tt==[]:
            tt=[x.TravelTime for x in y.VehTravelTimeList]
            while len(ts)<len(tt):
                ts.extend([y.StartTime])
        else:
            tt.extend([x.TravelTime for x in y.VehTravelTimeList])
            while len(ts)<len(tt):
                ts.extend([y.StartTime])


tt=[t-freett for t in tt]            
plt.plot(ts,tt,'b.',label='departure time')

plt.plot(stime,[c-s for c,s in zip(cycle,stage)],'r',label='cycle-green')
plt.plot(stime,[c for c,s in zip(cycle,stage)],'r--',label='cycle')
plt.xlabel('Cycle starting time')
plt.ylabel('Delay(s)')
plt.title('Delay Each Veh '+lgId)
plt.legend()
plt.savefig('SecondStop2.png',dpi=600,bbox_inches='tight')
plt.show()

In [18]:
def rmse(y_test, y):
    return sp.sqrt(sp.mean((y_test - y) * (y_test - y)))



def R2(y_test, y_true):
    return 1 - ((y_test - y_true) * (y_test - y_true)).sum() / (
    (y_true - y_true.mean()) * (y_true - y_true.mean())).sum()



def R22(y_test, y_true):
    y_mean = np.array(y_true)
    y_mean[:] = y_mean.mean()
    return 1 - rmse(y_test, y_true) / rmse(y_mean, y_true)

def intersectionpf(x, c, lam, s):
    v=x*s*lam
    for i in v:
        if i==0:
            i=0.1
    a=[math.pow(cc/vv/vv*3600*3600,1/3) for cc,vv in zip(c,v)]
    b=[math.pow(xx,2+5*llam) for xx,llam in zip(x,lam)]
    return c*(1-lam)*(1-lam)/2/(1-v/s)+x*x*1800/v/(1-x)-0.65*np.array(a)*np.array(b)


def intersectionpf2(x, c, lam, s, a):
    v=x*s*lam
    for i in v:
        if i==0:
            i=0.1
    return c*(1-lam)*(1-lam)/2/(1-v/s)+c*v*v/lam/s*5/3600


def intersectionpf3(x, a):
    s=3270
		#饱和流量veh/h
    v=x*s*lamda
    for i in v:
        if i==0:
            i=0.1
    c=np.array(cycle,dtype=int)
    lam=np.array(lamda,dtype=float)    
    return c*(1-lam)*(1-lam)/2/(1-v/s)+c*v*v/lam/s*a/3600


plt.figure(16,figsize=(8,4))
plt.plot(saturation,signaldelay,'b.',label='Data')
plt.plot(saturation, intersectionpf2(np.array(saturation,dtype=float),np.array(cycle,dtype=int),np.array(lamda,dtype=float),3270,10),'r.',label='Equation values')
plt.xlabel('Saturation Rate')
plt.ylabel('Signal Delay')
plt.title('Saturation Rate vs Delay '+lgId)
plt.legend()
plt.savefig('Saturation Rate vs Delay '+lgId+'.png',dpi=600,bbox_inches='tight')
plt.show()

In [19]:
plt.figure(17,figsize=(8,4))
plt.plot(signaldelay,'b.',label='Data')
error=str(R22(signaldelay,intersectionpf2(np.array(saturation,dtype=float),np.array(cycle,dtype=int),np.array(lamda,dtype=float),3270,10)))
plt.plot(intersectionpf2(np.array(saturation,dtype=float),np.array(cycle,dtype=int),np.array(lamda,dtype=float),3270,10),'r.',label='Equation values, a=10'+'\n error'+error)
plt.xlabel('Cycle index')
plt.ylabel('Signal Delay')
plt.title('Data vs Calculated '+lgId)
plt.savefig('Data vs Calculated '+lgId+'.png',dpi=600,bbox_inches='tight')
plt.legend()
plt.show()

In [20]:
from scipy.optimize import curve_fit
popt, pcov = curve_fit(intersectionpf3, saturation, signaldelay)
#print('a='+str(popt[0]))
#print('error='+str(R22(signaldelay,intersectionpf2(np.array(saturation,dtype=float),np.array(cycle,dtype=int),np.array(lamda,dtype=float),3270,popt[0]))))
a=str(popt[0])
error=str(R22(signaldelay,intersectionpf2(np.array(saturation,dtype=float),np.array(cycle,dtype=int),np.array(lamda,dtype=float),3270,popt[0])))


plt.figure(18,figsize=(8,4))
plt.plot(saturation,signaldelay,'b.',label='Data')
plt.plot(saturation,intersectionpf2(np.array(saturation,dtype=float),
                         np.array(cycle,dtype=int),
                         np.array(lamda,dtype=float),3270,popt[0]),'r.',label='Fitted values\n a='+a+'\n error='+error)
plt.xlabel('Saturation')
plt.ylabel('Signal Delay')
plt.title('Data vs Fitted')
plt.legend()
plt.savefig('Saturation-DelayData vs Fitted '+lgId+'.png',dpi=600,bbox_inches='tight')
plt.show()


plt.figure(19,figsize=(8,4))
plt.plot(signaldelay,'b.',label='Data')
plt.plot(intersectionpf2(np.array(saturation,dtype=float),
                         np.array(cycle,dtype=int),
                         np.array(lamda,dtype=float),3270,popt[0]),'r.',label='Fitted values\n a='+str(popt[0]))
plt.xlabel('Cycle index')
plt.ylabel('Signal Delay')
plt.title('Data vs Fitted')
plt.legend()
plt.savefig('Cycleindex-DelayData vs Fitted '+lgId+'.png',dpi=600,bbox_inches='tight')
plt.show()

def time_to_num(time_str):
    hh, mm , ss = map(int, str(time_str).split(':'))
    return ss + 60*(mm + 60*hh)

In [21]:
plt.figure(20,figsize=(8,4))
for y in cyleInfoList[index].CycleInfoList:
    if y.VehTravelTimeList:
        ttincycle=[time_to_num(((x.PreTimeStamp+timedelta(seconds=freett)).time())) for x in y.VehTravelTimeList]
        #print(ttincycle)
        travelTimeCount = len(ttincycle)
        if travelTimeCount<2 or (travelTimeCount==2 and ttincycle[0] == ttincycle[1]):
               pass    
        else:
            sns.kdeplot(np.array(ttincycle), cumulative=True, label=str(y.StartTime.time())+' arrival')      

for y in cyleInfoList[index].CycleInfoList:
    if y.VehTravelTimeList:
        #ttincycle2=[time_to_num((x.TimeStamp+timedelta(seconds=30)).time()) for x in y.VehTravelTimeList]
        ttincycle2=[time_to_num((x.TimeStamp).time()) for x in y.VehTravelTimeList]
        #print(ttincycle)
        travelTimeCount2 = len(ttincycle2)
        if travelTimeCount2<2 or (travelTimeCount2==2 and ttincycle2[0] == ttincycle2[1]):
               pass    
        else:
            sns.kdeplot(np.array(ttincycle2), cumulative=True, label=str(y.StartTime.time())+' departure')

plt.xlabel('Timestamp')
plt.ylabel('Frequency')
plt.title('Vehicle Arrival&Departure CDF'+lgId)
plt.savefig('Vehicle Arrival&Departure CDF '+lgId+'.png',dpi=600,bbox_inches='tight')
plt.show()

In [22]:
import pandas as pd
plt.figure(21,figsize=(20,4))
cmap = sns.blend_palette(["steelblue","yellow","red"], len(cyleInfoList[index].CycleInfoList)-1) 
sns.set_palette(cmap)
ttincycle3=[]
ttgroup=[]
with sns.color_palette(n_colors=len(cyleInfoList[index].CycleInfoList)): 
    for y in cyleInfoList[index].CycleInfoList:
        if y.VehTravelTimeList:
            travelTimeCount=len(y.VehTravelTimeList)
            tttt=[x.TravelTime for x in y.VehTravelTimeList]
            if travelTimeCount<2 or (travelTimeCount==2 and tttt[0] == [1]):
                pass
            else:
                ttincycle3.extend(tttt)
                ttgroup.extend([str(y.StartTime.time()) for x in y.VehTravelTimeList])

df = pd.DataFrame(dict(score=ttincycle3, group=ttgroup))            
sns.violinplot(df.group,df.score, scale='count')                    
plt.ylabel('Travel Time (s)')
plt.xlabel('Cycle labeled by stage starting time')
plt.title('Travel Time Distribution '+lgId)
plt.savefig('StageStart-Travel Time Distribution '+lgId+'.png',dpi=600,bbox_inches='tight')
plt.show()

In [23]:
plt.figure(22,figsize=(20,8))
cmap = sns.blend_palette(["steelblue","yellow","red"], len(cyleInfoList[index].CycleInfoList)-1) 
sns.set_palette(cmap)
ttincycle3=[]
ttgroup=[]
with sns.color_palette(n_colors=len(cyleInfoList[index].CycleInfoList)): 
    for y in cyleInfoList[index].CycleInfoList:
        if y.VehTravelTimeList:
            travelTimeCount=len(y.VehTravelTimeList)
            tttt=[time_to_num(((x.PreTimeStamp+timedelta(seconds=freett)).time())) for x in y.VehTravelTimeList]
            if travelTimeCount<2 or (travelTimeCount==2 and tttt[0] == [1]):
                pass
            else:
                ttincycle3.extend(tttt)
                ttgroup.extend([str(y.StartTime.time()) for x in y.VehTravelTimeList])

df = pd.DataFrame(dict(score=ttincycle3, group=ttgroup))            
sns.violinplot(df.group,df.score, scale='count')                    
plt.ylabel('Arrival time')
plt.xlabel('Cycle labeled by stage starting time')
plt.title('Arrival Time Distribution '+lgId)
plt.savefig('StageStart-Arrival Time Distribution '+lgId+'.png',dpi=600,bbox_inches='tight')
plt.show()